# Xinyu Xie's MSc Research Project Pipeline
Xinyu Xie's MSc Research Project Pipeline

This pipeline focuses on analyzing insect gut microbiome diversity and composition using 16S rRNA sequencing data. 
Explanation of how the different steps of the pipeline work can be found below the pipeline, in the 'Pipeline explanation' section.

Pipeline Steps:

1. Preparation for data download.
2. Downloading SRA data relevant data regarding insect gut microbiomes from the NCBI website.  
3. Converting SRA data to FASTQ format.
4. Sequencing data quality control and processing with FastQC software.
5. 16S rRNA gene sequencing analysis using QIIME.
       


## 1 - Preparation for data download
Before data download, Install essential tools like sratoolkit and Aspera Connect for efficient data downloading and handling.

01. Downloading SRA Toolkit

    https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit
    
    Download the installation package suitable for your computer model and double-click to extract it.

02. Installing Aspera Connect

    https://downloads.asperasoft.com/connect2/
    
    Click on "Download Aspera Connect" and follow the steps to download and install.
    Once downloaded, Aspera Connect will be enabled as a browser extension (you may need to manually adjust browser settings preferences to allow the extension to run).

03. Changing the Default Download Path of the SRA Toolkit

    Navigate to the bin folder in the sratoolkit: Enter

In [ ]:
cd /Users/SUO/Desktop/sratoolkit.3.1.0/bin/vdb-config -i

Use your keyboard to change the cache corresponding to the SRA data download path.

## 2 - Downloading SRA data and Converting SRA data to FASTQ format.
Downloading SRA data relevant data regarding insect gut microbiomes from the NCBI website.



01. Open a browser and enter the following URL:

    https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP017096&o=acc_s%3Aa
    
    This step uses project SRP017096 as example.

02. Download the "Accession List" to obtain the file named "SRR_Acc_List.txt"

03. Open the terminal and run the script download_srr.sh.

In [ ]:
#!/bin/bash

# define your SRR_Acc_List.txt path
ACC_LIST_PATH="/Users/SUO/Desktop/test/SRR_Acc_List.txt"

# define your output path
OUTPUT_DIR="/Users/SUO/Desktop/test/fastq"

# Check and create output directory if it doesn't exist
mkdir -p "$OUTPUT_DIR"

# read every single row of SRR number
while IFS= read -r srr_id
do
    echo "Downloading $srr_id"
    prefetch $srr_id
    # option: use fasterq-dump convert to FASTQ file
    echo "Converting $srr_id"
    fasterq-dump $srr_id -O "$OUTPUT_DIR"
done < "$ACC_LIST_PATH"

#get one specific SRR file to certain path:
#prefetch SRR616206 -O /Users/SUO/ncbi/SRP017096


The above step should convert SRR files into FASTQ files.

## 3 - Sequencing data quality control and processing with FastQC software.


01. Downloading conda environment

    Make sure your laptop has a conda environment : Miniconda or Anaconda
    (I already have Anaconda in my Macbook)

02. Installing Bioconda and Conda Forge channels


03. Install FastQC


In [ ]:
# update Conda
conda update conda

# Add Bioconda and Conda Forge channels
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge

# Install FastQC
conda install fastqc

# Check FastQC install
fastqc --version

04. Run run_fastqc.sh




In [ ]:
#!/bin/bash

# define input and output paths
INPUT_DIR="/Users/SUO/ncbi/test/fastq"
OUTPUT_DIR="/Users/SUO/ncbi/FASTQC"

# Check and create output directory if it doesn't exist
mkdir -p "$OUTPUT_DIR"

# read evry single FASTQ file and run FastQC
for fastq_file in "$INPUT_DIR"/*.fastq; do
    echo "Processing $fastq_file"
    fastqc -o "$OUTPUT_DIR" "$fastq_file"
done

echo "All files have been processed."

## 4 - 16S rRNA gene sequencing analysis using QIIME2:


## 4.1 - Download and use QIIME for 16S rRNA gene sequencing analysis

In [ ]:
conda update conda
conda install wget

# QIIME2 Amplicon Distribution for macOS(Apple Silicon)
wget https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.2-py38-osx-conda.yml
CONDA_SUBDIR=osx-64 conda env create -n qiime2-amplicon-2024.2 --file qiime2-amplicon-2024.2-py38-osx-conda.yml
conda activate qiime2-amplicon-2024.2
conda config --env --set subdir osx-64

# Test your installation
qiime --help

## 4.2 - Get manifest_file.csv to import fastq data into qiime2
    · Part 1: Generate file list CSV:

    Specify the directory path and generate a CSV file containing the filenames. 
    Traverse through all files in the directory and write the filenames into the CSV file.

    · Part 2: Generate manifest file:

    Use the generated filename CSV file as input. 
    Specify the output manifest file path and generate a manifest file containing the sample-id, absolute file path, and direction.

    
    · Part 3: Import data into QIIME2



In [ ]:
#!/bin/bash

# Enter the directory containing FASTQ files
cd /Users/SUO/ncbi/test/fastq || exit

# Create the manifest directory
mkdir -p manifest

# Create the manifest file header
{
    echo "# paired-end PHRED 33 fastq manifest file for forward and reverse reads"
    echo -e "sample-id\tforward-absolute-filepath\treverse-absolute-filepath"
} > manifest1.txt

# Ensure parallel is installed
if ! command -v parallel &> /dev/null
then
    echo "parallel could not be found, installing..."
    conda install -c conda-forge parallel
fi

# Create a text file containing IDs and paths for forward and reverse reads, separated by tabs
ls *.fastq | cut -d "_" -f 1 | sort | uniq | parallel -j0 --keep-order 'echo -e "{/}\t$PWD/{/}_1.fastq\t$PWD/{/}_2.fastq"' > manifest2.txt

# Merge the header file and content file to generate the final manifest file
cat manifest1.txt manifest2.txt > manifest/manifest.tsv

# Delete temporary text files
rm manifest1.txt manifest2.txt

# Return to the previous directory
cd ..


In [ ]:
# Initialize Conda
source ~/anaconda3/etc/profile.d/conda.sh

# Activate QIIME2 environment
conda activate qiime2-amplicon-2024.2

# Set the number of cores for parallel processing
NCORES=24

# Import sample data
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /Users/SUO/ncbi/test/fastq/manifest/manifest.tsv \
  --output-path demux.qza \
  --input-format PairedEndFastqManifestPhred33V2

# Check if the import was successful
if [ $? -ne 0 ]; then
    echo "Data import failed. Please check the manifest file and paths."
    exit 1
fi

# Generate a summary report of the data
qiime demux summarize --i-data demux.qza --o-visualization demux.qzv

# Check if the summary report was generated successfully
if [ $? -ne 0 ]; then
    echo "Summarize failed. Please check the input data."
    exit 1
fi

# Prompt to view the generated output report
echo "Data has been imported and summarized. You can view the output in demux.qzv."


## 4.3 - Sequence quality control and feature table


01. Trim data using Cutadapt

02. Denoise data using DADA2

03. Remove chimeras using Vsearch

04. OTU species classification and clustering with vsearch and Species taxonomic annotation

05. Phylogenetic tree construction

06. Diversity and composition analysis


    · Alpha diversity analysis: Assess the complexity of species diversity within each sample through metrics such as species richness and evenness.
    ?Shannon diversity index

    · Beta diversity analysis: Evaluate the differences in species composition between samples using statistical methods.
    ?Jaccard index
    ?Bray-Curtis dissimilarity matrix

    · summarize_taxa_through_plots.py: Use this QIIME script to summarize taxonomic composition and visualize the results in various plots.
    
    · OTU heat map and OTU network: Generate heat maps to visualize the relative abundance of OTUs across samples and create networks to visualize relationships between OTUs.


01. Trim data using Cutadapt

In [ ]:
#!/bin/bash

# Define the list of sample IDs
samples=("SRR6207228" "SRR6207229" "SRR6207226" "SRR6207235" "ERR4371932" "ERR4371938" "ERR4371941" "ERR3476225" "SRR8727637" "SRR8727632" "SRR8727636" "SRR8727634")

# Create output directory
output_dir="trimmed_sequences"
mkdir -p ${output_dir}

# Loop through each sample
for sample in "${samples[@]}"; do
  echo "Processing sample ${sample}..."

  # Define input and output file paths
  input_forward="/Users/SUO/ncbi/test/fastq/${sample}_1.fastq"
  input_reverse="/Users/SUO/ncbi/test/fastq/${sample}_2.fastq"
  output_forward="${output_dir}/${sample}_tr_R1.fastq.gz"
  output_reverse="${output_dir}/${sample}_tr_R2.fastq.gz"

  # Run Cutadapt for quality trimming
  cutadapt -q 20 -m 50 --trim-n \
    -o ${output_forward} -p ${output_reverse} \
    ${input_forward} ${input_reverse}

  echo "Finished processing sample ${sample}."
  echo "Output files: ${output_forward}, ${output_reverse}"
done

echo "All samples processed."


In [ ]:
#!/bin/bash

# Enter the directory containing FASTQ files
cd /Users/SUO/ncbi/test/fastq/0605/trimmed_sequences || exit

# Create manifest directory
mkdir -p tr_manifest

# Create the manifest file header
{
    echo "sample-id    forward-absolute-filepath    reverse-absolute-filepath"
} > tr_manifest1.txt

# Create a text file containing IDs and paths for forward and reverse reads
ls *.fastq.gz | cut -d "_" -f 1 | sort | uniq | while read sample; do
    forward="${PWD}/${sample}_tr_R1.fastq.gz"
    reverse="${PWD}/${sample}_tr_R2.fastq.gz"
    if [[ -f "$forward" && -f "$reverse" ]]; then
        echo -e "${sample}\t${forward}\t${reverse}"
    else
        echo "Warning: Files for sample $sample not found. Skipping."
    fi
done > tr_manifest2.txt

# Merge files
cat tr_manifest1.txt tr_manifest2.txt > tr_manifest/tr_manifest.tsv

# Return to the previous directory
cd -


02. Denoise data using DADA2

In [ ]:
# Import data
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /Users/SUO/ncbi/test/fastq/0605/trimmed_sequences/tr_manifest/tr_manifest.tsv \
  --output-path demux_trimmed.qza \
  --input-format PairedEndFastqManifestPhred33V2


# Perform denoising with DADA2
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux_trimmed.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 250 \
  --p-trunc-len-r 220 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza



In [ ]:
qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file /Users/SUO/ncbi/test/fastq/0605/trimmed_sequences/tr_manifest/tr_manifest.tsv


qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv


03. Remove chimeras using Vsearch

        Although the current proportion of non-chimeric sequences is relatively high, further manual inspection and verification can help ensure the accuracy of the results.

In [ ]:
# Run VSEARCH chimera detection 运行VSEARCH嵌合体检测
qiime vsearch uchime-denovo \
  --i-sequences rep-seqs.qza \
  --i-table table.qza \
  --o-chimeras chimeras.qza \
  --o-nonchimeras nonchimeras.qza \
  --o-stats chimera-stats.qza

# Generate a new feature table from non-chimeric sequences 从非嵌合体序列生成新的特征表
qiime feature-table filter-features \
  --i-table table.qza \
  --m-metadata-file nonchimeras.qza \
  --o-filtered-table filtered-table.qza

# Visualize the new feature table 可视化新的特征表
qiime feature-table summarize \
  --i-table filtered-table.qza \
  --o-visualization filtered-table.qzv \
  --m-sample-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv

# Visualize the feature table of chimeric sequences 可视化嵌合体序列的特征表
qiime feature-table filter-features \
  --i-table table.qza \
  --m-metadata-file chimeras.qza \
  --o-filtered-table chimeric-table.qza

qiime feature-table summarize \
  --i-table chimeric-table.qza \
  --o-visualization chimeric-table.qzv \
  --m-sample-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv

# Visualize chimera detection statistics 可视化嵌合体检测统计信息
qiime metadata tabulate \
  --m-input-file chimera-stats.qza \
  --o-visualization chimera-stats.qzv


04. OTU species classification and clustering with vsearch

In [ ]:
#  Run VSEARCH clustering 运行VSEARCH聚类
qiime vsearch cluster-features-de-novo \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --p-perc-identity 0.97 \
  --o-clustered-table clustered-table.qza \
  --o-clustered-sequences clustered-seqs.qza

# Visualize the clustered feature table 可视化聚类后的特征表
qiime feature-table summarize \
  --i-table clustered-table.qza \
  --o-visualization clustered-table.qzv \
  --m-sample-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv

# Visualize the clustered representative sequences 可视化聚类后的代表序列
qiime feature-table tabulate-seqs \
  --i-data clustered-seqs.qza \
  --o-visualization clustered-seqs.qzv


In [ ]:
#Download Silva training set for taxonomy annotation
wget -c https://data.qiime2.org/2023.5/common/silva-138-99-nb-classifier.qza


# Taxonomy annotation
qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-nb-classifier.qza \
  --i-reads clustered-seqs.qza \
  --o-classification taxonomy.qza

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

qiime taxa barplot \
  --i-table clustered-table.qza  \
  --i-taxonomy taxonomy.qza \
  --o-visualization taxa-bar-plots.qzv


05. hylogenetic tree construction

In [ ]:

# Step 1: Multiple Sequence Alignment
qiime alignment mafft \
  --i-sequences clustered-seqs.qza \
  --o-alignment aligned-rep-seqs.qza

# Step 2: Masking
qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

# Step 3: Construct Unrooted Tree
qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza

# Step 4: Root the Tree
qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

qiime tools export \
  --input-path rooted-tree.qza \
  --output-path exported-tree

06. Diversity and composition analysis


    · Alpha diversity analysis: Assess the complexity of species diversity within each sample through metrics such as species richness and evenness.
    ?Shannon diversity index

    · Beta diversity analysis: Evaluate the differences in species composition between samples using statistical methods.
    ?Jaccard index
    ?Bray-Curtis dissimilarity matrix

    
    · OTU heat map and OTU network: Generate heat maps to visualize the relative abundance of OTUs across samples and create networks to visualize relationships between OTUs.

当没有生成系统发育树时仍可进行核心多样性的计算，运行代码如下：
Core diversity calculation can still be performed even if a phylogenetic tree is not generated. Run the code as follows:
qiime diversity core-metrics--i-table table.qza--p-sampling-depth xxx--m-metadata-file metadata.tsv --output-dir outdir

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table clustered-table.qza \
  --p-sampling-depth 2000 \
  --m-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv \
  --output-dir core-metrics-results


The output files contain:
输出结果文件 (13个数据文件):

- core-metrics-results/faith_pd_vector.qza: Alpha多样性考虑进化的faith指数；  
  (Alpha diversity Faith's phylogenetic diversity index)

- core-metrics-results/unweighted_unifrac_distance_matrix.qza: 无权重unifrac距离矩阵；  
  (Unweighted UniFrac distance matrix)

- core-metrics-results/bray_curtis_pcoa_results.qza: 基于Bray-Curtis距离PCoA的结果；  
  (PCoA results based on Bray-Curtis distance)

- core-metrics-results/shannon_vector.qza: Alpha多样性香农指数；  
  (Alpha diversity Shannon index)

- core-metrics-results/rarefied_table.qza: 等量重采样后的特征表；  
  (Feature table after rarefaction)

- core-metrics-results/weighted_unifrac_distance_matrix.qza: 有权重的unifrac距离矩阵；  
  (Weighted UniFrac distance matrix)

- core-metrics-results/jaccard_pcoa_results.qza: jaccard距离PCoA结果；  
  (PCoA results based on Jaccard distance)

- core-metrics-results/observed_otus_vector.qza: Alpha多样性observed otus指数；  
  (Alpha diversity observed OTUs index)

- core-metrics-results/weighted_unifrac_pcoa_results.qza: 基于有权重的unifrac距离PCoA结果；  
  (PCoA results based on weighted UniFrac distance)

- core-metrics-results/jaccard_distance_matrix.qza: jaccard距离矩阵；  
  (Jaccard distance matrix)

- core-metrics-results/evenness_vector.qza: Alpha多样性均匀度指数；  
  (Alpha diversity evenness index)

- core-metrics-results/bray_curtis_distance_matrix.qza: Bray-Curtis距离矩阵；  
  (Bray-Curtis distance matrix)

- core-metrics-results/unweighted_unifrac_pcoa_results.qza: 无权重的unifrac距离的PCoA结果。  
  (PCoA results based on unweighted UniFrac distance)

Output files (4 types of visualizations):
输出对象 (4种可视化结果):

- core-metrics-results/unweighted_unifrac_emperor.qzv: 无权重的unifrac距离PCoA结果采用emperor可视化；  
  (PCoA results based on unweighted UniFrac distance visualized using Emperor)

- core-metrics-results/jaccard_emperor.qzv: jaccard距离PCoA结果采用emperor可视化；  
  (PCoA results based on Jaccard distance visualized using Emperor)

- core-metrics-results/bray_curtis_emperor.qzv: Bray-Curtis距离PCoA结果采用emperor可视化；  
  (PCoA results based on Bray-Curtis distance visualized using Emperor)

- core-metrics-results/weighted_unifrac_emperor.qzv: 有权重的unifrac距离PCoA结果采用emperor可视化。  
  (PCoA results based on weighted UniFrac distance visualized using Emperor)

Alpha多样性组间显著性分析和可视化命令：
Command for significance analysis and visualization of alpha diversity between groups:

In [ ]:

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv \
  --o-visualization core-metrics-results/faiths_pd_statistics.qzv

用方差分析（ANOVA）来测试多重效应是否显着影响α多样性
Use analysis of variance (ANOVA) to test whether multiple factors significantly affect alpha diversity

In [ ]:
qiime longitudinal anova
--m-metadata-file core-metrics-results/faith_pd_vector.qza
--m-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv \
--p-formula 'faith_pd ~ genotype * donor_status' 
--o-visualization core-metrics-results/faiths_pd_anova.qzv


Beta多样性组间显著性分析和可视化
Significance analysis and visualization of beta diversity between groups

In [ ]:
qiime diversity beta-group-significance 
--i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza
--m-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv 
--m-metadata-column group 
--o-visualization core-metrics-results/unweighted-unifrac-cage-significance.qzv
--p-pairwise


检查group相关的差异是否是由于group内较大差异引起
Check if the differences related to the group are caused by significant differences within the group

In [ ]:
qiime diversity beta-group-significance
--i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza
--m-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv 
--m-metadata-column group 
--o-visualization core-metrics-results/unweighted-unifrac-cage-significance_disp.qzv
--p-method permdisp

Alpha稀疏和深度选择 Alpha Rarefaction and Selecting a Rarefaction Depth

In [ ]:
 qiime diversity alpha-rarefaction
--i-table clustered-table.qza 
--i-phylogeny rooted-tree.qza 
--p-max-depth 4250
--m-metadata-file /Users/SUO/ncbi/test/fastq/trimmed_sequences/tr_manifest/tr_manifest.tsv
--o-visualization alpha-rarefaction.qzv
